In [1]:
%matplotlib inline
import sys
sys.path.append("/media/bianyuan/data-TNG-1/Grab_TNGgalaxy/")
# sys.path.append("./Grab_TNGgalaxy")
import illustris_python as il
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
%matplotlib widget
run = 'TNG50'
basePath = '/media/bianyuan/data-TNG-1/' + run + '-1/output'

In [94]:
GroupFirstSub = il.groupcat.loadHalos(basePath,99,fields=['GroupFirstSub'])
GroupNsubs = il.groupcat.loadHalos(basePath,99,fields=['GroupNsubs'])


In [73]:
'''Tranfer SnapNum to Redshift'''

import numpy as np
import h5py, scipy
from pynbody import units as units_conv

Redshift_snapshot = np.loadtxt('/media/bianyuan/data-TNG-1/data/Redshift_snapshot.txt',comments='Snapshot')

def redshift_snapshot(snapshots, datafile='/media/bianyuan/data-TNG-1/data/Redshift_snapshot.txt', return_opt=None):
    snapshots0, redshifts0 = np.loadtxt(datafile, skiprows=1, unpack=True, usecols=[0, 2])
    n_snap = np.size(snapshots)
    if n_snap == 1: 
        redshifts = redshifts0[np.where(snapshots0 == snapshots)][0]
    elif n_snap >1:
        redshifts = np.zeros(n_snap)
        for ii in range(n_snap): 
            if  snapshots[ii] is np.nan: 
                redshifts[ii] = np.nan
            else:
                redshifts[ii] = redshifts0[np.where(snapshots0 == snapshots[ii])][0]
    if return_opt is None: 
        return redshifts
#   elif return_opt == 'age':
#       return age(redshifts)
    elif return_opt == 'FT':
        return FT(redshifts)

def _a_dot(a, h0, om_m, om_l):                                
    om_k = 1.0 - om_m - om_l      
    return h0 * a * np.sqrt(om_m * (a ** -3) + om_k * (a ** -2) + om_l)  

def _a_dot_recip(*args):
    return 1. / _a_dot(*args) 

# The formation time of stars in unit of Gyr
def FT(redshift, h0=0.6774, OmegaM=0.3089, OmegaL=0.6911):
    conv = units_conv.Unit("0.01 s Mpc km^-1").ratio('Gyr')
    redshift = 1./(1. + redshift)
    ns = np.size(redshift)
    if ns > 1:
        FT = np.zeros(ns)
        for ii in range(ns): FT[ii] = scipy.integrate.quad(_a_dot_recip, 0, redshift[ii], (h0, OmegaM, OmegaL))[0] * conv
    else:
        FT = scipy.integrate.quad(_a_dot_recip, 0, redshift, (h0, OmegaM, OmegaL))[0] * conv
    return FT

In [100]:
fields = ['SubhaloMassInHalfRadType','SnapNum','SubhaloStarMetallicity','SubhaloSFRinHalfRad','SubhaloHalfmassRadType','SubhaloStarMetallicityHalfRad','GroupFirstSub','SubhaloGrNr']

tree = il.sublink.loadTree(basePath,99,947737,fields=fields,onlyMPB=True)

print(GroupFirstSub[tree['SubhaloGrNr'][0]])

947737


In [80]:
Htree = il.lhalotree.treeOffsets(basePath,99,947737)
print(Htree)

(0, 218296839, 0)


In [91]:
# basePath = '/media/bianyuan/data-TNG-1/' + run + '-1/postprocessing'
fields = ['FirstHaloInFOFGroup','NextHaloInFOFGroup','SubhaloNumber','FirstProgenitor']

Halotree = il.lhalotree.loadTree(basePath,99,63864,fields=fields,onlyMPB=True)

In [92]:
print(Halotree['FirstHaloInFOFGroup'])

[64120718 64120717 64120716 64120715 64120714 64120713 64120712 64120711
 64120710 64120709 64120708 64120707 64120706 64120705 64120704 64120703
 64120702 64120701 64120700 64120997 64120996 64120995 64120994 64122444
 64122443 64122442 64122441 64122440 64122439 64122438 64127593 64127592
 64127591 64129491 64129490 64129489 64129488 64129487 64129486 64129485
 64129484 64129483 64129482 64129481 64129480 64129479 64129478 64129477
 64129476 64129475 64129474 64129473 64129472 64129471 64129470 64129469
 64129468 64129467 64129466 64129465 64129464 64129463 64129462 64129461
 64129460 64129459 64412984 64412983 64412982 64412981 64525360 64525359
 64525358 64525393 64525392 65344265 65344266 65344267 65344268 65344269
 65344270 82406182 65344272 65344273 65344274 65344275 65344276 65344277
 65344278 65344279 65344280 65344281 65344282 65344283 65344284 65344285
 65344286 65344287 65344288 65344289]


In [46]:
import matplotlib as mpl
dm_pos = il.snapshot.loadSubset(basePath,99,'dm',['Coordinates']);
plt.hist2d(dm_pos[:,0], dm_pos[:,1], norm=mpl.colors.LogNorm(), bins=64);

MemoryError: Unable to allocate 225. GiB for an array with shape (10077696000, 3) and data type float64